In [50]:
import pandas as pd
from pathlib import Path
import math
import pprint

ORBIT_DATA_DIR  = Path(r"E:\interim\step2_normalized_test")
EQ_PATH    = Path(r"E:\tables\orbit_quake_ver4.csv")
OUTPUT_DIR      = Path(r"E:\tables")

print("orbit data directory =", {ORBIT_DATA_DIR})
print("Orbit_meet_time_condion.csv path=", {EQ_PATH})
print("Directory of output", {OUTPUT_DIR})

# WGS84 model
a = 6378137                                                             # the long radius of the Earth
b = 6356752.314245                                                      # the short radius of the Earth

e = math.sqrt((a*a - b*b)/(a*a))                                    # eccentricity of the Earth

print("a:<type>{},", type(a), a)
print("b:<type>{},", type(b), b)
print("e:<type>{}", type(e), e)


orbit data directory = {WindowsPath('E:/interim/step2_normalized_test')}
Orbit_meet_time_condion.csv path= {WindowsPath('E:/tables/orbit_quake_ver4.csv')}
Directory of output {WindowsPath('E:/tables')}
a:<type>{}, <class 'int'> 6378137
b:<type>{}, <class 'float'> 6356752.314245
e:<type>{} <class 'float'> 0.08181919084296535


In [53]:
df = pd.read_csv(EQ_PATH)
df_ob = pd.read_csv(r"E:\interim\step2_normalized_test\DMT_N1_1132_00197.1_step1_step2.csv")

list1 =[[] for k in range(len(df_ob))]

for i in range(len(df_ob)):
    midlist = []
    lat1 = df["lat"].iloc[0] * math.pi/180
    lon1 = df["lon"].iloc[0] * math.pi/180
    print("<type>lat1:", type(lat1), lat1)
    print("<type>lon1:", type(lon1), lon1)

    lat2 = df_ob["lat"].iloc[i] * math.pi/180
    lon2 = df_ob["lon"].iloc[i] * math.pi/180
    if lon2 > math.pi:
        lon2 = lon2 - 2*math.pi

    print("<type>lat2:", type(lat2), lat2)
    print("<type>lon2:", type(lon2), lon2)

    dif_lat = lat2 - lat1
    dif_lon = lon2 - lon1
        
    P = (lat1+lat2) / 2.0                                         # 両点緯度の平均値
    W = math.sqrt(1.0-e*e * math.sin(P) * math.sin(P))
    M = (a*(1.0 - e*e)) / (W * W * W)
    N = a / W

    t1 = M * dif_lat
    t2 = N * dif_lon * math.cos(P)

    dist = math.sqrt(t1 ** 2 + t2 ** 2)
    dist = dist / 1000                                          # 単位[km] => [m]

    midlist.append(dist)

    list1[i] = midlist

data = pd.DataFrame(list1)
outputone = pd.concat([df_ob, data], axis=1)
outputone.to_csv(OUTPUT_DIR / "dist_of_each_sample_test.csv")

    

<type>lat1: <class 'numpy.float64'> 0.820898160383013
<type>lon1: <class 'numpy.float64'> 2.6890462318401833
<type>lat2: <class 'numpy.float64'> -0.9611978877484048
<type>lon2: <class 'numpy.float64'> -0.36048921357865193
<type>lat1: <class 'numpy.float64'> 0.820898160383013
<type>lon1: <class 'numpy.float64'> 2.6890462318401833
<type>lat2: <class 'numpy.float64'> -0.9591048095791705
<type>lon2: <class 'numpy.float64'> -0.36158270752223753
<type>lat1: <class 'numpy.float64'> 0.820898160383013
<type>lon1: <class 'numpy.float64'> 2.6890462318401833
<type>lat2: <class 'numpy.float64'> -0.9570117314099365
<type>lon2: <class 'numpy.float64'> -0.3626762014658249
<type>lat1: <class 'numpy.float64'> 0.820898160383013
<type>lon1: <class 'numpy.float64'> 2.6890462318401833
<type>lat2: <class 'numpy.float64'> -0.9539947360795265
<type>lon2: <class 'numpy.float64'> -0.36425199388282437
<type>lat1: <class 'numpy.float64'> 0.820898160383013
<type>lon1: <class 'numpy.float64'> 2.6890462318401833
<typ

In [ ]:
df = pd.read_csv(EQ_PATH)

orbit_file_name = list(ORBIT_DATA_DIR.glob("*.csv"))
pprint.pprint(orbit_file_name)

list1 = [[] for k in range(len(orbit_file_name))]

for i in range(len(orbit_file_name)):
    midlist = []

    orbit_data_path = ORBIT_DATA_DIR / orbit_file_name[i]

    df_ob = pd.read_csv(orbit_data_path)
    
    for j in range(len(df_ob)):
        lat1 = df["lat"].iloc[i] * math.pi/180
        lon1 = df["lon"].iloc[i] * math.pi/180
        print("<type>lat1:", type(lat1), lat1)
        print("<type>lon1:", type(lon1), lon1)

        lat2 = df_ob["lat"].iloc[j] * math.pi/180
        lon2 = df_ob["lon"].iloc[j] * math.pi/180
        if lon2 > math.pi:
            lon2 = lon2 - 2*math.pi

        print("<type>lat2:", type(lat2), lat2)
        print("<type>lon2:", type(lon2), lon2)

        dif_lat = lat2 - lat1
        dif_lon = lon2 - lon1
            
        P = (lat1+lat2) / 2                                         # 両点緯度の平均値
        W = math.sqrt((1-e*e * math.sin(P) * math.sin(P)))
        M = (a*(1 - e*e)) / (W * W * W)
        N = a / W

        dist = math.sqrt(dif_lat*dif_lat*M*M + dif_lon*dif_lon*N*N*math.cos(P)*math.cos(P))
        dist = dist / 1000                                          # 単位[km] => [m]

        midlist.append(dist)

    list1[i] = midlist

pprint.pprint(list1)

data = pd.DataFrame(list1)
file_name = pd.DataFrame(orbit_file_name)
output_one = pd.concat([file_name, data], axis=1)
output_one.to_csv(OUTPUT_DIR / "dist_orbit_test.csv")